This workbook is a workaround as the AEMDA data is incorrect (TechIDs are mixed up).

This workbook takes the numveh95 table (from TEAM-Kenya) and the technology table. It 'refreshes' the techIDs.

*WARNING WARNING* this is quick + dirty. Waiting for Paschal's analysis.

In [9]:
import pandas as pd

In [10]:
numveh95 = pd.read_csv(f'./ccg_team_notebook/out_data/processed/numveh95.csv')
numveh95

,CountryID,TechID,Age,TransTypeID,VehTypeID,MassCatID,EngineID,FuelID,NumVeh
0,9,70,5,1,3,1,1,1,8.469686e+01
1,9,70,6,1,3,1,1,1,1.693937e+02
2,9,70,7,1,3,1,1,1,5.928780e+02
3,9,70,8,1,3,1,1,1,6.775749e+02
4,9,70,9,1,3,1,1,1,9.316654e+02
...,...,...,...,...,...,...,...,...,...
5205,9,4275,16,1,8,45,7,15,9.028010e-02
5206,9,4275,32,1,8,45,7,15,3.025660e-12
5207,9,4275,33,1,8,45,7,15,3.782154e-16
5208,9,4275,34,1,8,45,7,15,2.250437e-21


In [11]:
technology = pd.read_excel(f'./ccg_team_notebook/sample_data/team_kenya_technology.xlsx')
technology

,TechID,ModeID,VehTypeID,MassCatID,FuelID,EngineID,TransTypeID,Availability,Final_Year,HybridFlag,spec_energ_consump,spec_energ_electric,noise_class,PurchasePrice,ExpectedLifeFactor,Capacity,AveEconLife,SecondHandImportFlag
0,10,1,2,4,1,10,1,1980,2000.0,0,6.571951,NaN,0,5000.0,1,2.0,10,0
1,20,1,2,4,12,10,1,1995,2010.0,0,9.514442,NaN,2,7000.0,1,2.0,10,0
2,25,1,2,4,12,11,1,2010,2020.0,0,8.380919,NaN,2,6000.0,1,2.0,10,0
3,26,1,2,4,12,12,1,2020,NaN,0,7.183646,NaN,2,5500.0,1,2.0,10,0
4,30,1,2,4,1,11,1,2000,2010.0,0,5.645645,NaN,0,5000.0,1,2.0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,14889,1,5,28,13,13,2,2033,2038.0,0,179.200000,1.0,2,14000.0,1,2.5,10,1
2242,14890,1,5,28,13,13,2,2038,2043.0,0,153.890000,1.0,2,12000.0,1,2.5,10,1
2243,14891,1,5,28,13,13,2,2043,2048.0,0,132.150000,1.0,2,12000.0,1,2.5,10,1
2244,14892,1,5,28,13,13,2,2048,2053.0,0,113.480000,1.0,2,12000.0,1,2.5,10,1


In [12]:
for i in list(numveh95.index.values):

    # fetch parameters from this row
    techid, transtype, vehtype, masscat, fuel, engine = numveh95.TechID[i], numveh95.TransTypeID[i], numveh95.VehTypeID[i], numveh95.MassCatID[i], numveh95.FuelID[i], numveh95.EngineID[i]

    # return matched records in technology
    tech = technology[(technology.TransTypeID == transtype) & (technology.VehTypeID == vehtype) & (technology.MassCatID == masscat) & (technology.FuelID == fuel) & (technology.EngineID == engine)]

    if techid not in tech.TechID.tolist():  # then there is a mismatch

        if not tech.empty:

            if techid > 9000 and not tech[tech.TechID > 9000].empty:
                numveh95.at[i, 'TechID'] = tech[tech.TechID > 9000].TechID.tolist()[0]

            else:
                numveh95.at[i, 'TechID'] = tech.TechID.tolist()[0]

        else:  # it is empty so find a simpler match

            tech_simple = technology[(technology.VehTypeID == vehtype) & (technology.MassCatID == masscat) & (technology.FuelID == fuel)]

            if not tech_simple.empty:

                if techid > 9000 and not tech_simple[tech_simple.TechID > 9000].empty:
                    numveh95.at[i, 'TechID'] = tech_simple[tech_simple.TechID > 9000].TechID.tolist()[0]

                else:
                    numveh95.at[i, 'TechID'] = tech_simple.TechID.tolist()[0]

            else:  # then god help us all just go for vehicle and fuel

                tech_super_simple = technology[(technology.VehTypeID == vehtype) & (technology.FuelID == fuel)]

                if not tech_super_simple.empty:

                    if techid > 9000 and not tech_super_simple[tech_super_simple.TechID > 9000].empty:
                        numveh95.at[i, 'TechID'] = tech_super_simple[tech_super_simple.TechID > 9000].TechID.tolist()[0]

                    else:
                        numveh95.at[i, 'TechID'] = tech_super_simple.TechID.tolist()[0]

                else:  # then wtf

                    numveh95.at[i, 'TechID'] = technology[technology.VehTypeID==vehtype].TechID.tolist()[0]

numveh95

,CountryID,TechID,Age,TransTypeID,VehTypeID,MassCatID,EngineID,FuelID,NumVeh
0,9,70,5,1,3,1,1,1,8.469686e+01
1,9,70,6,1,3,1,1,1,1.693937e+02
2,9,70,7,1,3,1,1,1,5.928780e+02
3,9,70,8,1,3,1,1,1,6.775749e+02
4,9,70,9,1,3,1,1,1,9.316654e+02
...,...,...,...,...,...,...,...,...,...
5205,9,4275,16,1,8,45,7,15,9.028010e-02
5206,9,4275,32,1,8,45,7,15,3.025660e-12
5207,9,4275,33,1,8,45,7,15,3.782154e-16
5208,9,4275,34,1,8,45,7,15,2.250437e-21


In [13]:
numveh95.to_csv(f'./ccg_team_notebook/out_data/processed/afema_bodge/numveh95_afemabodge.csv')